In [18]:
import intake
import satstac
import yaml

import intake_stac
intake_stac.__version__

'0.3.0.post58'

# Load an Intake Catalog

In [60]:
url = 'https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/catalog.json'
#url = 'https://planetarycomputer.microsoft.com/api/stac/v1'
#url = 'https://raw.githubusercontent.com/intake/intake-stac/main/intake_stac/tests/data/1.0.0/catalog/catalog.json'
stac_cat = intake.open_stac_catalog(url)
list(stac_cat['extensions-collection'])

['proj-example']

In [75]:
a = stac_cat['extensions-collection']['proj-example']['B1']
print(a)

sources:
  stac_asset:
    args:
      chunks: {}
      urlpath: https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF
    description: Band 1 (coastal)
    driver: intake_xarray.raster.RasterIOSource
    metadata:
      catalog_dir: ''
      eo:bands:
      - center_wavelength: 0.44
        common_name: coastal
        full_width_half_max: 0.02
        name: B1
      href: https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF
      plots:
        geotiff:
          cmap: viridis
          data_aspect: 1
          dynamic: true
          frame_width: 500
          kind: image
          rasterize: true
          x: x
          y: y
      title: Band 1 (coastal)
      type: image/tiff; application=geotiff



# Serialize as a .YAML file

In [32]:
#stac_cat.walk(depth=2) # Works! type dict
#stac_cat.walk(depth=10).serialize() # Attribute Error, no attribute `serialize`

#print(stac_cat.yaml()) # Works! type str
#stac_cat.yaml(depth=1) # Type Error, unexpected keyword `depth`, this is our target api

#print(stac_cat.test.yaml()) # type st
#print(stac_cat.test) #type StacCatalog

In [33]:
#yaml.dump(stac_cat.walk(depth=2))
#print(yaml.dump(stac_cat)) # type str
#print(yaml.dump(stac_cat.walk(depth=0))) # type str

#Missing source: and shouldn't have !!python/object:intake.catalog.local.LocalCatalogEntry to work

In [36]:
with open('test10.yaml', 'w') as f:
    f.write(yaml.dump(stac_cat.walk(depth=10)))

Code from: https://github.com/intake/intake/blob/471488801bfa338dbfd91f9851b27580f4f3c3bb/intake/source/base.py#L242-L265showing how intake turns its catalogues into a dictionary with very specific keys before into a yaml.

```
def _yaml(self):
    import inspect
    kwargs = self._captured_init_kwargs.copy()
    meta = kwargs.pop('metadata', self.metadata) or {}
    kwargs.update(dict(zip(inspect.signature(self.__init__).parameters,
                       self._captured_init_args)))
    data = {
        'sources':
            {self.name: {
                'driver': self.classname,
                'description': self.description or "",
                'metadata': meta,
                'args': kwargs
            }}}
    return data

def yaml(self):
    """Return YAML representation of this data-source
    The output may be roughly appropriate for inclusion in a YAML
    catalog. This is a best-effort implementation
    """
    data = self._yaml()
    return yaml.dump(data, default_flow_style=False)
```

In [99]:
import inspect
import intake_xarray
kwargs = a._captured_init_kwargs.copy()
meta = kwargs.pop('metadata', a.metadata) or {}
kwargs.update(dict(zip(inspect.signature(a.__init__).parameters,
                a._captured_init_args)))

In [102]:
a.classname

'intake_xarray.raster.RasterIOSource'

In [100]:
#`a` is a dictionary of `RasterIOSource`s. I can pull out some attributes, but not others. Unsure why.a
intake_drivers = [
    'netcdf = intake_xarray.netcdf:NetCDFSource',
    'zarr = intake_xarray.xzarr:ZarrSource',
    'opendap = intake_xarray.opendap:OpenDapSource',
    'xarray_image = intake_xarray.image:ImageSource',
    'rasterio = intake_xarray.raster:RasterIOSource',
    'remote-xarray = intake_xarray.xarray_container:RemoteXarray'
]

name = a.name

driver = a.classname #how to get short one from list?
#if isinstance(a, intake_xarray.raster.RasterIOSource):
#    driver = 'rasterio' # recreate to get from all drivers.

description = a.description #will be branching, different behavior for sub catalog

metadata = a.metadata

args = kwargs

In [104]:
a._yaml() #for datasources (end of the branch) existing yaml implimentation works. Go up one level now

{'sources': {'stac_asset': {'driver': 'intake_xarray.raster.RasterIOSource',
   'description': 'Band 1 (coastal)',
   'metadata': {'href': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF',
    'type': 'image/tiff; application=geotiff',
    'title': 'Band 1 (coastal)',
    'eo:bands': [{'name': 'B1',
      'common_name': 'coastal',
      'center_wavelength': 0.44,
      'full_width_half_max': 0.02}],
    'plots': {'geotiff': {'kind': 'image',
      'x': 'x',
      'y': 'y',
      'frame_width': 500,
      'data_aspect': 1,
      'rasterize': True,
      'dynamic': True,
      'cmap': 'viridis'}},
    'catalog_dir': ''},
   'args': {'urlpath': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF',
    'chunks': {}}}}}

In [95]:
#I'm trying to recreate the data dictionary from intake.yaml() without the references to self, but since driver can't be accessed it is stuck.

#data = {
#        'sources':
#            {self.name: {
#                'driver': self.classname,
#                'description': self.description or "",
#                'metadata': meta,
#                'args': kwargs
#            }}}

data = {
    'sources':
        {name: {
            'driver': driver,
            'description': description,
            'metadata': metadata,
            'args': args
            }}}

example-catalog:
  extensions-collection:
    proj-example:
      B1:
        sources: ...
        description: ...

In [ ]:
b = stac_cat['extensions-collection']['proj-example']
#b._yaml()
print(b.serialize())

In [ ]:
y = {'metadata':{'version':1}, 'sources':{}}
for key, source in stac_cat['extensions-collection']['proj-example'].items():
    y['sources'][key] = source._yaml()

print(yaml.dump(y))
#does B1 ever have multiple sources, might not work when loaded because of extra layer of nesting, prune this dict manually

In [ ]:
c =  stac_cat['extensions-collection']
c._yaml()

# Full Circle: Load Intake Catalogue from .YAML

In [13]:
catalog = intake.open_catalog('works.yaml') # Constructor Error

In [ ]:
catalog2 = intake.open_catalog('test.yaml') # Constructor Error